In [1]:
server_args = []
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

from manipulation import running_as_notebook

# Imports
import numpy as np
import pydot
from ipywidgets import Dropdown, Layout
from IPython.display import display, HTML, SVG

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, autoDiffToGradientMatrix, SnoptSolver,
    initializeAutoDiffGivenGradientMatrix, autoDiffToValueMatrix, autoDiffToGradientMatrix,
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint
)

In [2]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)

In [3]:
#add table
ground = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/ground.urdf"), "ground")

#add box
cracker_box = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/003_cracker_box_test.urdf"), "cracker_box")


In [4]:
plant.Finalize()
plant.set_name("plant")

In [5]:
cracker_box_body = plant.GetBodyByName("base_link_cracker", cracker_box)
cracker_box_id = plant.GetBodyFrameIdIfExists(cracker_box_body.index())


draw_frames = True
frames_to_draw = [cracker_box_id] if draw_frames else []
meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, frames_to_draw=frames_to_draw, axis_length=0.15,
                 axis_radius=0.001,)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
# X_WB = RigidTransform(RotationMatrix(), [0.55, 0., 0.3285 + 0.03295])
# plant.SetFreeBodyPose(plant_context, 
#                               plant.GetBodyByName("base_link_cracker", cracker_box),
#                               X_WB)

meshcat.load()
diagram.Publish(context)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


In [6]:
# q0_o = plant.GetPositions(plant_context, cracker_box)
p = plant.CalcPointsPositions(plant_context, plant.GetFrameByName('point_collision5'), [0,0,0.0334], plant.world_frame())[2]
print(p)

[0.]


In [7]:
# Need this because a==b returns True even if a = AutoDiffXd(1, [1, 2]), b= AutoDiffXd(2, [3, 4])
# That's the behavior of AutoDiffXd in C++, also.
def autoDiffArrayEqual(a,b):
    return np.array_equal(a, b) and np.array_equal(autoDiffToGradientMatrix(a), autoDiffToGradientMatrix(b))

mu = 0.5 
total_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(cracker_box))
gravity = plant.gravity_field().gravity_vector()
collision_point_frame = [
    plant.GetFrameByName('point_collision5'),
    plant.GetFrameByName('point_collision6'),
    plant.GetFrameByName('point_collision7'),
    plant.GetFrameByName('point_collision8')]

#set contact sequence
N = 100
# in_contact = np.zeros((N))
# in_contact[40:] = 1

T = 1
prog = MathematicalProgram()
# Time steps    
h = prog.NewContinuousVariables(N-1, "h")
prog.AddBoundingBoxConstraint(0.5*T/N, 2.0*T/N, h) 
prog.AddLinearConstraint(sum(h) >= .9*T)
prog.AddLinearConstraint(sum(h) <= 1.1*T)

# Create one context per timestep (to maximize cache hits)
plant_context_list = [plant.CreateDefaultContext() for i in range(N)]
# We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
ad_plant = plant.ToAutoDiffXd()

nq_o = plant.num_positions(cracker_box)
nv_o = plant.num_velocities(cracker_box)
q_o = prog.NewContinuousVariables(nq_o, N, "q_o")
v_o = prog.NewContinuousVariables(nv_o, N, "v_o")
vdot_o = prog.NewContinuousVariables(nv_o, N, "vdot_o")

#initial constrain
q0_o = plant.GetPositions(plant_context, cracker_box)
v0_o = plant.GetVelocities(plant_context, cracker_box)
prog.AddBoundingBoxConstraint(q0_o, q0_o, q_o[:,0])
prog.AddBoundingBoxConstraint(v0_o, v0_o, v_o[:,0])
# prog.AddBoundingBoxConstraint([0]*nv_o, [0]*nv_o, vdot_o[:,0])
for n in range(N):
    prog.SetInitialGuess(q_o[:,n], q0_o)  
    # Unit quaternions
    AddUnitQuaternionConstraintOnPlant(plant, q_o[:,n], prog)
    # Running costs:
#     prog.AddQuadraticErrorCost(np.diag([1]*nq_o), q0_o, q_o[:,n])
#     prog.AddQuadraticErrorCost(np.diag([1]*nv_o), [0]*nv_o, v_o[:,n])
#     prog.AddQuadraticErrorCost(np.diag([10]*nv_o), [0]*nv_o, vdot_o[:,n])

    
# Make a new autodiff context for this constraint (to maximize cache hits)
ad_plant_context_list = [ad_plant.CreateDefaultContext() for i in range(N)]
def velocity_position_constraint(vars, context_index):
    h, q, v, qn = np.split(vars, [1, 1+nq_o, 1+nq_o+nv_o])
    if isinstance(vars[0], AutoDiffXd):
        if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetPositions(ad_plant_context_list[context_index], q)
        v_from_qdot = ad_plant.MapQDotToVelocity(ad_plant_context_list[context_index], (qn - q)/h)
    else:
        if not np.array_equal(q, plant.GetPositions(plant_context_list[context_index])):
            plant.SetPositions(plant_context_list[context_index], q)
        v_from_qdot = plant.MapQDotToVelocity(plant_context_list[context_index], (qn - q)/h)
    return v - v_from_qdot
for n in range(N-1):
    prog.AddConstraint(
        partial(velocity_position_constraint, context_index=n), 
        lb=[0]*nv_o, ub=[0]*nv_o, 
        vars=np.concatenate(([h[n]], q_o[:,n], v_o[:,n+1], q_o[:,n+1])))  #(implicit Euler)
    
#accelaration and velocity relation constrain(implicit Euler)
for n in range(N-1):
#     prog.AddConstraint(eq(q_o[:, n+1], q_o[:,n] + h[n]*v_o[:,n]))
    prog.AddConstraint(eq(v_o[:, n+1], v_o[:,n] + h[n]*vdot_o[:,n+1]))

def calc_phiq(vars, context_index):
    q, f_n = np.split(vars, [nq_o])
      # get reference frames for collision_point
    ad_collision_point_frame = [
    ad_plant.GetFrameByName('point_collision5'),
    ad_plant.GetFrameByName('point_collision6'),
    ad_plant.GetFrameByName('point_collision7'),
    ad_plant.GetFrameByName('point_collision8')]
    if isinstance(vars[0], AutoDiffXd):
        if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetPositions(ad_plant_context_list[context_index], q)
        phiq = [ad_plant.CalcPointsPositions(ad_plant_context_list[context_index], ad_collision_point_frame[collision_point], [0,0,0], ad_plant.world_frame())[2] for collision_point in range(4)]
        phiq = np.array(phiq).flatten()
        phiq = phiq + 0.0334
        
        f_n_dot_phiq = [phiq[collision_point]*f_n[collision_point] for collision_point in range(4)]
        res = np.concatenate((phiq, f_n_dot_phiq))
    else:
        if not np.array_equal(q, plant.GetPositions(plant_context_list[context_index])):
            plant.SetPositions(plant_context_list[context_index], q)
        phiq = [plant.CalcPointsPositions(plant_context_list[context_index], collision_point_frame[collision_point], [0,0,0], plant.world_frame())[2] for collision_point in range(4)]
        phiq = np.array(phiq).flatten()
        phiq = phiq + 0.0334
        
        f_n_dot_phiq = [phiq[collision_point]*f_n[collision_point] for collision_point in range(4)]
        res = np.concatenate((phiq, f_n_dot_phiq))
    return res   
epsilon = 1e-2
#relate to box frame
collision_point_contact_force = [prog.NewContinuousVariables(3, N, f"collision_point{collision_point}_contact_force") for collision_point in range(4)]
#why only to N-1
for n in range(N):
    vars = np.concatenate((q_o[:, n], 
                        [collision_point_contact_force[0][2, n]], [collision_point_contact_force[1][2, n]],
                        [collision_point_contact_force[2][2, n]], [collision_point_contact_force[3][2, n]]))
    prog.AddConstraint(
        partial(calc_phiq, context_index=n), 
        lb=np.concatenate(([0]*4, [0]*4)), ub=np.concatenate(([np.inf]*4, [epsilon]*4)), 
        vars=vars)
    for collision_point in range(4):
        # Linear friction cone
        prog.AddLinearConstraint(collision_point_contact_force[collision_point][0,n] <= mu*collision_point_contact_force[collision_point][2,n])
        prog.AddLinearConstraint(-collision_point_contact_force[collision_point][0,n] <= mu*collision_point_contact_force[collision_point][2,n])
        prog.AddLinearConstraint(collision_point_contact_force[collision_point][1,n] <= mu*collision_point_contact_force[collision_point][2,n])
        prog.AddLinearConstraint(-collision_point_contact_force[collision_point][1,n] <= mu*collision_point_contact_force[collision_point][2,n])
        # normal force >=0, normal_force == 0 if not in_contact
        prog.AddBoundingBoxConstraint(0, 4*9.81*total_mass, collision_point_contact_force[collision_point][2,n])


def manipulator_equations(vars):
    nf = 3
    # split input vector in subvariables
    # configuration, velocity, acceleration, collision_point force
    split_at = [nq_o, nq_o + nv_o, nq_o + 2 * nv_o, nq_o + 2 * nv_o + nf, nq_o + 2 * nv_o + 2 * nf, nq_o + 2 * nv_o + 3 * nf]
    q, v, vdot, f1, f2, f3, f4 = np.split(vars, split_at)
    f = [f1, f2, f3, f4]

    ad_plant_context = ad_plant.CreateDefaultContext()
    ad_plant.SetPositions(ad_plant_context, q)
    ad_plant.SetVelocities(ad_plant_context, v)
    
    # matrices for the manipulator equations
    M = ad_plant.CalcMassMatrixViaInverseDynamics(ad_plant_context)
    Cv = ad_plant.CalcBiasTerm(ad_plant_context)
    tauG = ad_plant.CalcGravityGeneralizedForces(ad_plant_context)

    
    # Jacobian of the collision_point
    # get reference frames for collision_point
    ad_collision_point_frame = [
    ad_plant.GetFrameByName('point_collision5'),
    ad_plant.GetFrameByName('point_collision6'),
    ad_plant.GetFrameByName('point_collision7'),
    ad_plant.GetFrameByName('point_collision8')]

    # compute Jacobian matrix
    J = [ad_plant.CalcJacobianTranslationalVelocity(
        ad_plant_context,
        JacobianWrtVariable.kV,
        ad_collision_point_frame[collision_point],
        [0., 0., 0.],
        ad_plant.world_frame(),
        ad_plant.world_frame())
        for collision_point in range(4)]
    

    tauF = np.zeros((nv_o))
    for collision_point in range(4):
        tauF = tauF + J[collision_point].T.dot(f[collision_point])
        
    
    # return violation of the manipulator equations
    return M.dot(vdot) + Cv - tauG - tauF

# manipulator equations for all t (implicit Euler)
for n in range(N):
    vars = np.concatenate((q_o[:, n], v_o[:, n], vdot_o[:, n], 
                           collision_point_contact_force[0][:, n], collision_point_contact_force[1][:, n],
                           collision_point_contact_force[2][:, n], collision_point_contact_force[3][:, n]))
    prog.AddConstraint(manipulator_equations, lb=[0]*nv_o, ub=[0]*nv_o, vars=vars)

In [8]:
snopt = SnoptSolver().solver_id()
prog.SetSolverOption(snopt, 'Iterations Limits', 1e5 if running_as_notebook else 1)
prog.SetSolverOption(snopt, 'Major Iterations Limit', 200 if running_as_notebook else 1)
prog.SetSolverOption(snopt, 'Major Feasibility Tolerance', 5e-6)
prog.SetSolverOption(snopt, 'Major Optimality Tolerance', 1e-4)
prog.SetSolverOption(snopt, 'Superbasics limit', 2000)
prog.SetSolverOption(snopt, 'Linesearch tolerance', 0.9)
#prog.SetSolverOption(snopt, 'Print file', 'snopt.out')



result = Solve(prog)
print(result.get_solver_id().name())
if not result.is_success():
#     print(result.GetInfeasibleConstraintNames(prog))
    print("solve failed!")
else:
    print("solve success")
    q_o_sol = result.GetSolution(q_o)
    print("q_o_sol:", q_o_sol)

SNOPT/fortran
solve failed!


In [8]:
v_o_sol = result.GetSolution(v_o)
print("v_o_sol:", v_o_sol)

v_o_sol: [[ 0.00000000e+00  0.00000000e+00  2.06795153e-25  2.06795153e-25
   2.06795153e-25  3.10192730e-25  3.10192730e-25  3.10192730e-25
   3.10192730e-25  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.03397577e-25
  -1.03397577e-25 -1.03397577e-25 -1.03397577e-25 -1.03397577e-25
  -1.03397577e-25 -1.03397577e-25 -1.03397577e-25 -1.03397577e-25
  -1.03397577e-25 -1.03397577e-25 -1.03397577e-25 -1.03397577e-25
  -1.03397577e-25 -1.03397577e-25 -1.03397577e-25 -1.03397577e-25
 

In [9]:
vdot_o_sol = result.GetSolution(vdot_o)
print("vdot_o_sol:", vdot_o_sol)

vdot_o_sol: [[ 6.01971318e-03  0.00000000e+00  9.92616735e-24  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+0

In [10]:
collision_point_contact_force_sol = [result.GetSolution(collision_point_contact_force[collision_point]) for collision_point in range(4)]

print("collision_point0_contact_force:", collision_point_contact_force_sol[0] + collision_point_contact_force_sol[1] +
     collision_point_contact_force_sol[2] + collision_point_contact_force_sol[3])


collision_point0_contact_force: [[ 1.76173329e-03 -2.77555756e-17 -1.39956559e-03  2.77555756e-17
   0.00000000e+00  0.00000000e+00 -5.55111512e-17  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.77555756e-17
  -2.77555756e-17 -2.77555756e-17  5.55111512e-17 -2.77555756e-17
  -2.77555756e-17 -2.77555756e-17 -2.77555756e-17  0.00000000e+00
  -2.77555756e-17 -2.77555756e-17 -2.77555756e-17 -2.77555756e-17
   0.00000000e+00  5.95369276e-16  0.00000000e+00  0.00000000e+00
   1.62658991e-03  1.62647510e-03  1.11022302e-16 -2.49800181e-16
  -2.77555756e-17  3.60822483e-16  8.32667268e-17 -2.77555756e-17
   1.38777878e-16 -8.32667268e-17 -2.77555756e-17  2.77555756e-17
   5.55111512e-17 -2.22044605e-16  2.77555756e-17 -2.49800181e-16
   5.55111512e-17  1.66533454e-16 -1.11022302e-16  2.22044605e-16
   8.32667268e-17  5.55111512e-17 -1.38777878e-16 -5.55111512e-17
  -2.77555756e-17  5.55111512e-17 -1.11022302e-16 -1.66533454e-16
   1.38777878e-16  1.62453981e-03 -8.3266726

In [14]:
print("collision_point0_contact_force:", collision_point_contact_force_sol[3]
     )

collision_point0_contact_force: [[-1.84712706e-01  0.00000000e+00  0.00000000e+00  2.16182462e-01
   2.16182462e-01  2.16182462e-01  1.33069336e-01  2.16182462e-01
   2.16182462e-01  2.16182462e-01  2.16182462e-01  0.00000000e+00
   2.16182462e-01  0.00000000e+00  1.33069336e-01  0.00000000e+00
   0.00000000e+00  1.33069336e-01  2.16182462e-01  2.16182462e-01
   1.33069336e-01  2.16182462e-01  2.16182462e-01  0.00000000e+00
   0.00000000e+00 -1.52533873e-17  2.16182462e-01  2.16182462e-01
   1.98223893e-04  1.98209903e-04  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.87114923e-01  1.87114923e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.87114923e-01
   0.00000000e+00  0.00000000e+00  1.87114923e-01  0.00000000e+00
   1.87114923e-01  1.97974059e-04  0.0000000

In [12]:
# manipulator equations for all t (implicit Euler)
for n in range(N-1):
    vars = np.concatenate((q_o_sol[:, n], v_o_sol[:, n], vdot_o_sol[:, n], 
                           collision_point_contact_force_sol[0][:, n], collision_point_contact_force_sol[1][:, n],
                           collision_point_contact_force_sol[2][:, n], collision_point_contact_force_sol[3][:, n]))
    print("manipulator_equations:", autoDiffToValueMatrix(manipulator_equations(vars)))

manipulator_equations: [[ 2.43937866e-16]
 [ 1.83559663e-16]
 [ 1.33204401e-17]
 [ 9.71445147e-17]
 [ 6.24500451e-17]
 [-2.44249065e-15]]
manipulator_equations: [[ 4.11476409e-15]
 [-6.60929644e-15]
 [ 7.25808302e-15]
 [-8.32667268e-17]
 [ 0.00000000e+00]
 [-3.33066907e-16]]
manipulator_equations: [[-5.26488575e-16]
 [-3.53883589e-16]
 [ 7.37257477e-18]
 [-9.89208715e-14]
 [ 4.85722573e-17]
 [-1.11022302e-16]]
manipulator_equations: [[ 4.12170298e-15]
 [-6.59541866e-15]
 [ 7.26935873e-15]
 [ 1.11022302e-16]
 [ 2.77555756e-17]
 [ 2.22044605e-16]]
manipulator_equations: [[ 4.11476409e-15]
 [-6.59194921e-15]
 [ 7.26241983e-15]
 [-1.11022302e-16]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
manipulator_equations: [[ 4.10782519e-15]
 [-6.58501031e-15]
 [ 7.25981775e-15]
 [-2.77555756e-17]
 [ 0.00000000e+00]
 [ 5.55111512e-16]]
manipulator_equations: [[ 4.12864187e-15]
 [-6.61276589e-15]
 [ 7.26935873e-15]
 [ 1.38777878e-16]
 [-2.77555756e-17]
 [ 1.11022302e-16]]
manipulator_equations: [[ 4.121702

In [16]:
print(q0_o)
print(v0_o)

[1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]


In [17]:
def manipulator_equations_val(vars):
    nf = 3
    # split input vector in subvariables
    # configuration, velocity, acceleration, collision_point force
    split_at = [nq_o, nq_o + nv_o, nq_o + 2 * nv_o, nq_o + 2 * nv_o + nf, nq_o + 2 * nv_o + 2 * nf, nq_o + 2 * nv_o + 3 * nf]
    q, v, vdot, f1, f2, f3, f4 = np.split(vars, split_at)
    f = [f1, f2, f3, f4]

    plant_context = plant.CreateDefaultContext()
    plant.SetPositions(plant_context, q)
    plant.SetVelocities(plant_context, v)
    
    # matrices for the manipulator equations
    M = plant.CalcMassMatrixViaInverseDynamics(plant_context)
    Cv = plant.CalcBiasTerm(plant_context)
    tauG = plant.CalcGravityGeneralizedForces(plant_context)
    print("tauG:\n" , tauG)
    print("g:\n" , plant.gravity_field().gravity_vector())
    # Jacobian of the collision_point
    # get reference frames for collision_point
    collision_point_frame = [
    plant.GetFrameByName('point_collision5'),
    plant.GetFrameByName('point_collision6'),
    plant.GetFrameByName('point_collision7'),
    plant.GetFrameByName('point_collision8')]

    # compute Jacobian matrix
    J = [plant.CalcJacobianTranslationalVelocity(
        plant_context,
        JacobianWrtVariable.kV,
        collision_point_frame[collision_point],
        [0., 0., 0.],
        plant.world_frame(),
        plant.world_frame())
        for collision_point in range(4)]
    

    tauF = np.zeros((nv_o))
    for collision_point in range(4):
        tauF = tauF + J[collision_point].T.dot(f[collision_point])
        print("J", collision_point, ": \n" , J[collision_point])
        
    print("#######################")
    # return violation of the manipulator equations
    return M.dot(vdot) + Cv - tauG - tauF

In [18]:
for n in range(1):
    vars = np.concatenate((q_o_sol[:, n], v_o_sol[:, n], vdot_o_sol[:, n], 
                           collision_point_contact_force_sol[0][:, n], collision_point_contact_force_sol[1][:, n],
                           collision_point_contact_force_sol[2][:, n], collision_point_contact_force_sol[3][:, n]))
    print("manipulator_equations:", manipulator_equations_val(vars))

tauG:
 [-0.    -0.    -0.    -0.    -0.    -0.981]
g:
 [ 0.    0.   -9.81]
J 0 : 
 [[ 0.     -0.0334 -0.1042  1.      0.      0.    ]
 [ 0.0334  0.      0.0795  0.      1.      0.    ]
 [ 0.1042 -0.0795  0.      0.      0.      1.    ]]
J 1 : 
 [[ 0.     -0.0334  0.1042  1.      0.      0.    ]
 [ 0.0334  0.      0.0795  0.      1.      0.    ]
 [-0.1042 -0.0795  0.      0.      0.      1.    ]]
J 2 : 
 [[ 0.     -0.0334 -0.1042  1.      0.      0.    ]
 [ 0.0334  0.     -0.0795  0.      1.      0.    ]
 [ 0.1042  0.0795  0.      0.      0.      1.    ]]
J 3 : 
 [[ 0.     -0.0334  0.1042  1.      0.      0.    ]
 [ 0.0334  0.     -0.0795  0.      1.      0.    ]
 [-0.1042  0.0795  0.      0.      0.      1.    ]]
#######################
manipulator_equations: [0.0000000e+00 6.9388939e-18 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00]


In [19]:
p_WF = plant.CalcPointsPositions(plant_context, plant.GetFrameByName('point_collision5'), [0,0,0], plant.world_frame())
print(p_WF)
X_WF = plant.CalcRelativeTransform(plant_context, plant.GetFrameByName('point_collision5'), plant.world_frame())
print(X_WF)

[[0.]
 [0.]
 [0.]]
RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)


In [19]:
print(gravity)

[ 0.    0.   -9.81]
